#### UTILITARY SCRIPT TO CONVERT IMAGES IN AGGREGATED DATA

In [2]:
import pandas as pd
import numpy as np
import json
import itertools
import copy


Taille (en m2) des données compilées :

In [3]:
DIM_AGREGATION = 25 # X 25 meters

Import initial list of parcelles :

In [4]:
df = pd.read_excel('IFN/data_parcelles_with_gps.xlsx')

In [5]:
df_base = df[['PARCELLE','LFI']]

----------------

#### 1) To Import and to convert a unique JSON File of satellites images :

In [5]:
LFI = 3 # define the campaign

First_parcelle = '51' #the name of the first parcelle of the dict

In [18]:
Extension = '' #in the case of problematics parcelles file, to fill with expression like "_pb1"

In [16]:
with open(f"Forest-Protection-Satellite-Images/Results_Images_Stock/Images_LANDSAT_LFI{LFI}{Extension}.json", 'r') as openfile: 
    data = json.load(openfile)

Verifications (lenght, LFI and dims) :

In [17]:
len(data)

240

Try with a parcelle:

In [ ]:
data[First_parcelle]['LFI']

'LFI4'

In [ ]:
np.shape(data[First_parcelle]['IMAGES_SAT']['NDVI'])

(200, 200)

LFI Agregation (if error : correspondance false)

In [ ]:
LFI = data[First_parcelle]['LFI']
df_result = df_base.loc[df_base['LFI']==LFI,:].reset_index(drop=True)

Script and loop :

In [ ]:
nb_parc = len(df_result)
problematic_parcelles = []
list_parcelle = df_result['PARCELLE'].tolist()
width = np.shape(data[First_parcelle]['IMAGES_SAT']['NDVI'])[0]
height = np.shape(data[First_parcelle]['IMAGES_SAT']['NDVI'])[1]
range_w = np.arange(int((width - DIM_AGREGATION)/2), int((width + DIM_AGREGATION)/2))
range_h = np.arange(int((height - DIM_AGREGATION)/2), int((height + DIM_AGREGATION)/2))
ndvi_arr = np.empty(nb_parc)
evi_arr = np.empty(nb_parc)
ndmi_arr = np.empty(nb_parc)
ndwi_arr = np.empty(nb_parc)
dswi_arr = np.empty(nb_parc)
MAPPING = {
    'NDVI' : ndvi_arr,
    'EVI' : evi_arr,
    'NDMI' : ndmi_arr,
    'NDWI' : ndwi_arr,
    'DSWI' : dswi_arr
}

for ind, parc in enumerate(list_parcelle):
    try:
        for func_name, arr in MAPPING.items():
            pixel_values = data[str(parc)]['IMAGES_SAT'][func_name]
            arr[ind] = round(np.mean([pixel_values[tupl[0]][tupl[1]] for tupl in list(itertools.product(range_w,range_h))]),4)
    except:
        problematic_parcelles.append(parc)
        for _ , arr in MAPPING.items():
            arr[ind] = np.nan

df_result['NDVI'] = ndvi_arr
df_result['EVI'] = evi_arr
df_result['NDMI'] = ndmi_arr
df_result['NDWI'] = ndwi_arr
df_result['DSWI'] = dswi_arr

Verify number of parcelles with "NaN" data :

In [ ]:
df_result['DSWI'].isnull().sum()

2221

In [ ]:
len(problematic_parcelles)

2221

Export :

In [ ]:
df_result.to_excel(f'./DATA_aggregated/Data_from_satellites_images_{LFI}_pb1.xlsx')

------------------

#### 2) To Import and to convert multiple JSON Files of satellites images :

Parameters :

In [7]:
nb_parts = 10 # numbers of files

LFI_name = 'LFI3'

LFI = 3

width = 200 # of initial images

height = 200 # of initial images


Loop for conversion in a list of 'nb_parts' dataframes...

In [8]:
df_result_base = df_base.loc[df_base['LFI']==LFI_name,:].reset_index(drop=True)
dataframes_results = []
nb_parc = len(df_result_base)
list_parcelle = df_result_base['PARCELLE'].tolist()
range_w = np.arange(int((width - DIM_AGREGATION)/2), int((width + DIM_AGREGATION)/2))
range_h = np.arange(int((height - DIM_AGREGATION)/2), int((height + DIM_AGREGATION)/2))


for i in range(nb_parts):
    try:
        with open(f"Forest-Protection-Satellite-Images/Results_Images_Stock/Images_LANDSAT_LFI{LFI}_part{i+1}.json", 'r') as openfile:
            data = json.load(openfile)
        print(f'Opening file part {i+1} with lenght {len(data)} ...')

        empty_parcelles = []
        df_result = copy.copy(df_result_base)
        ndvi_arr = np.empty(nb_parc)
        evi_arr = np.empty(nb_parc)
        ndmi_arr = np.empty(nb_parc)
        ndwi_arr = np.empty(nb_parc)
        dswi_arr = np.empty(nb_parc)
        MAPPING = {
            'NDVI' : ndvi_arr,
            'EVI' : evi_arr,
            'NDMI' : ndmi_arr,
            'NDWI' : ndwi_arr,
            'DSWI' : dswi_arr
        }

        print('Conversion and aggregation...')
        try:
            for ind, parc in enumerate(list_parcelle):
                try:
                    for func_name, arr in MAPPING.items():
                        pixel_values = np.empty(2)
                        pixel_values = data[str(parc)]['IMAGES_SAT'][func_name]
                        arr[ind] = round(np.mean([pixel_values[tupl[0]][tupl[1]] for tupl in list(itertools.product(range_w,range_h))]),4)
                except:
                    empty_parcelles.append(parc)
                    for _ , arr in MAPPING.items():
                        arr[ind] = np.nan
        except:
            print(f'Conversion\'s problem with file part {i+i} ...')

        print('Creation of a dataframe...')
                
        df_result['NDVI'] = ndvi_arr
        df_result['EVI'] = evi_arr
        df_result['NDMI'] = ndmi_arr
        df_result['NDWI'] = ndwi_arr
        df_result['DSWI'] = dswi_arr

        dataframes_results.append(df_result)

        print(f"Difference of empty data for NDVI : {len(empty_parcelles) - df_result['NDVI'].isnull().sum()}")
        print(f"Difference of empty data for EVI : {len(empty_parcelles) - df_result['EVI'].isnull().sum()}")
        print(f"Difference of empty data for NDMI : {len(empty_parcelles) - df_result['NDMI'].isnull().sum()}")
        print(f"Difference of empty data for NDWI : {len(empty_parcelles) - df_result['NDWI'].isnull().sum()}")
        print(f"Difference of empty data for DSWI : {len(empty_parcelles) - df_result['DSWI'].isnull().sum()}")

    except:
        print(f'Opening problem with file part {i+i} ...')
    print('Done...')
print('Finish...')              

Opening file part 1 with lenght 240 ...
Conversion and aggregation...
Creation of a dataframe...
Difference of empty data for NDVI : 0
Difference of empty data for EVI : 0
Difference of empty data for NDMI : 0
Difference of empty data for NDWI : 0
Difference of empty data for DSWI : 0
Done...
Opening file part 2 with lenght 240 ...
Conversion and aggregation...
Creation of a dataframe...
Difference of empty data for NDVI : 0
Difference of empty data for EVI : 0
Difference of empty data for NDMI : 0
Difference of empty data for NDWI : 0
Difference of empty data for DSWI : 0
Done...
Opening file part 3 with lenght 240 ...
Conversion and aggregation...
Creation of a dataframe...
Difference of empty data for NDVI : 0
Difference of empty data for EVI : 0
Difference of empty data for NDMI : 0
Difference of empty data for NDWI : 0
Difference of empty data for DSWI : 0
Done...
Opening file part 4 with lenght 240 ...
Conversion and aggregation...
Creation of a dataframe...
Difference of empty d

Join with the 'n_parts' dataframes ...

In [9]:
nb_lines = len(df_result_base)
result_arr = np.empty((nb_lines,5))

for i in range(nb_parts):
    c=0
    arr = np.empty((nb_lines,5))
    arr = dataframes_results[i].iloc[:,2:].to_numpy()
    for line in range(nb_lines):
        for col in range(5):
            if str(arr[line][col]) != 'nan':
                result_arr[line][col] = copy.copy(arr[line][col])
                c+=1
    print(f'Join n°{i} : {c} data copied ...')
    
    

Join n°0 : 1200 data copied ...
Join n°1 : 1200 data copied ...
Join n°2 : 1200 data copied ...
Join n°3 : 1200 data copied ...
Join n°4 : 1200 data copied ...
Join n°5 : 1200 data copied ...
Join n°6 : 1200 data copied ...
Join n°7 : 1200 data copied ...
Join n°8 : 1200 data copied ...
Join n°9 : 1215 data copied ...


Creation of the datafame result :

In [10]:
df_result = copy.copy(df_result_base)
df_result[['NDVI','EVI','NDMI','NDWI','DSWI']] = result_arr
df_result

,PARCELLE,LFI,NDVI,EVI,NDMI,NDWI,DSWI
0,51,LFI3,0.6923,0.0272,0.2294,-0.6305,0.9579
1,384,LFI3,0.3752,0.0148,0.0619,-0.3731,0.4551
2,1239,LFI3,0.3704,0.0109,0.1901,-0.3553,0.5237
3,1419,LFI3,0.3690,0.0100,0.2203,-0.3510,0.5608
4,1431,LFI3,0.1174,0.0028,0.0660,-0.1507,0.1888
...,...,...,...,...,...,...,...
2398,164918,LFI3,0.5413,0.0146,0.1105,-0.4951,0.6097
2399,164922,LFI3,0.5173,0.0180,0.1398,-0.4851,0.6109
2400,164999,LFI3,0.5339,0.0158,0.0742,-0.5077,0.5778
2401,165003,LFI3,0.5174,0.0173,0.1501,-0.4745,0.6091


Test with a parcelle :

In [11]:
df_result.loc[df_result['PARCELLE']==12298,:]

,PARCELLE,LFI,NDVI,EVI,NDMI,NDWI,DSWI
37,12298,LFI3,0.4262,0.0138,0.1572,-0.3929,0.4992


In [12]:
df_result.to_excel(f'Forest-Protection-Satellite-Images/Data_from_satellites_images_{LFI_name}.xlsx')